In [8]:
import numpy as np
import gensim
import nltk
import re
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
import pandas as pd
from nltk.tokenize import sent_tokenize
import multiprocessing
import os
from pathlib import Path
import io
from gensim.models import Phrases
from textblob import TextBlob
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

In [9]:
# 3개의 입력: 전공 3개
majors = """ mechanical engineering, Film Studies, Psychology  """
major_list_ = majors.split(",")
#소문자로 변환
major_list = []
for mjl in major_list_:
    mjl_ = mjl.lower()
    mjl__ = mjl_.strip()
    major_list.append(mjl__)

major_list

['mechanical engineering', 'film studies', 'psychology']

In [10]:

def major_analysis(text):

    input_corpus = str(text) #문장입력
    input_corpus = input_corpus.lower()#소문자 변환

    sentences  = sent_tokenize(input_corpus) #문장 토큰화

    split_sentences = []
    for sentence in sentences:
        processed = re.sub("[^a-zA-Z]"," ", sentence)
        words = processed.split()
        split_sentences.append(words)
                
    return split_sentences

In [11]:
def activity_anaysis(text):

    input_corpus = str(text) #문장입력
    input_corpus = input_corpus.lower()#소문자 변환

    sentences  = sent_tokenize(input_corpus) #문장 토큰화

    split_sentences = []
    for sentence in sentences:
        processed = re.sub("[^a-zA-Z]"," ", sentence)
        words = processed.split()
        split_sentences.append(words)
    result = sum(split_sentences, [])
    return result

In [12]:
# 일단 3개의 EXTRACURRICULAR ACTIVITY EXAMPLES  입력, 추가로 활동을 입력할 수 있음. 최대 10개, 그 이상도 가능하지만 비율로 게산

input_text_1 = """ deputy Member (9th/10th) Treasurer (11th/12th) National Honors Society, Ridgefield High School Chapter
We are amongst the highest academically achieving students at our school, who collectively and consistently participate in community service projects.""" # 실제 값은 문장이 입력되어야 함, 현재는 테스트용 단어입력

input_text_2 = """ Leader/Concertmaster (10th-12th) unite for sight
AMAC Youth Chamber Ensemble (AYCE), audition-based community choir 
Lead ensemble in rehearsal and performance, stand now coordinate rehearsal times, aid younger  """

input_text_3 = """ Number 1 Doubles Starter (9th-12th), Captain (11th-12th)
JV/V Beverly Hills High School Tennis Team
Three year League Champions; planned and hosted team banquet; led team warmups and meetings; Coach's Award Recipient (11th); Team Spirit Award (12th).  """

input_text_4 = """ Peer Advisor (11th-12th)
Erving High School Student Ambassador Program, Psychology selective application-based leadership team mechanical engineering
Organized and led orientation; served as a year round leader, mentor, tutor, and friend to freshmen; helped with class scheduling."""

input_text_5 = """ Leader (11th)
Books on Global Health Equity and Social Justice, advocacy-focused peer discussion group
Researched global health equity/social justice , assigned weekly readings for group discussion, brainstormed questions to generate input from members.  """

input_text_6 = """ Number 1 Doubles Starter (9th-12th), Captain (11th-12th)
JV/V Beverly Hills High School Tennis Team
Three year League Champions; planned and hosted team banquet; led team warmups and meetings; Coach's Award Recipient (11th); Team Spirit Award (12th).  """

input_text_7 = """ Number 1 Doubles Starter (9th-12th), Captain (11th-12th)
JV/V Beverly Hills High School Tennis Team
Three year League Champions; planned and hosted team banquet; led team warmups and meetings; Coach's Award Recipient (11th); Team Spirit Award (12th).  """

input_text_8 = """Student Coach (9th - 12th)
Middle School MathCounts Team
Taught strategies, selected competitors, hosted weekly practice sessions and lectures. Led team to 2nd place victory at State Mathematics competition (11th). """

input_text_9 = """ Protein Modeling Team Leader (10th)
Science Olympiad, Burke High School Club
Supervised building of protein molecule model, taught peers to use 3D molecular program Jmol; placed in top ten in 2017 regional competition. """

input_text_10 = """""" #이것은 값이 없기 때문에 null로 처리해 보자

In [13]:
# 2. 아래 2개의 함수가 실행된다. 


print('major_list : ', major_list)
print("======================================")
activity_list = activity_anaysis(input_text_4)
print('activity_list :', activity_list)

major_list :  ['mechanical engineering', 'film studies', 'psychology']
activity_list : ['peer', 'advisor', 'th', 'th', 'erving', 'high', 'school', 'student', 'ambassador', 'program', 'psychology', 'selective', 'application', 'based', 'leadership', 'team', 'mechanical', 'engineering', 'organized', 'and', 'led', 'orientation', 'served', 'as', 'a', 'year', 'round', 'leader', 'mentor', 'tutor', 'and', 'friend', 'to', 'freshmen', 'helped', 'with', 'class', 'scheduling']


In [14]:
#데이터 불러오기 
data_major = pd.read_csv('total_major_fit_dataset.csv')
data_major[:10]

,Unnamed: 0,major,main_major
0,0,mechanical engineering,tech_eng
1,1,general engineering,tech_eng
2,2,mechanical engineering,tech_eng
3,3,industrial engineering,tech_eng
4,4,bioengineering,tech_eng
5,5,operations research,tech_eng
6,6,materials science,tech_eng
7,7,electrical engineering,tech_eng
8,8,computer science/engineering,tech_eng
9,9,computer engineeringcivil engineering,tech_eng


In [15]:
#major 3개와 연관된 major category를 추출해본다. 우선 데이터 확인
category_major = data_major['major']
category_major

0      mechanical engineering
1         general engineering
2      mechanical engineering
3      industrial engineering
4              bioengineering
                ...          
314                    debate
315                    speech
316                 newspaper
317                     media
318             communication
Name: major, Length: 319, dtype: object

In [16]:
data_major['main_major'] #데이터 확인

0           tech_eng
1           tech_eng
2           tech_eng
3           tech_eng
4           tech_eng
           ...      
314    communication
315    communication
316    communication
317    communication
318    communication
Name: main_major, Length: 319, dtype: object

In [17]:
'film studies' in data_major['major'].values

True

In [18]:
extract_major = []
for mj in major_list:
    if mj in data_major['major'].values: # 매칭되는 값이 있다면, 해당 행을 찾아서
        
        ext_index = data_major[data_major['major'] == mj] #조건에 맞는 값을 모두 가져온다.

        extract_major.append(ext_index)
        
extract_major

[   Unnamed: 0                   major main_major
 0           0  mechanical engineering   tech_eng
 2           2  mechanical engineering   tech_eng,
      Unnamed: 0         major  main_major
 185           1  film studies  humanities
 227          43  film studies  humanities,
      Unnamed: 0       major  main_major
 124           2  psychology  social_sci
 156          34  psychology  social_sci]

In [19]:
#extract_major 를 문자로 만들어서 개별 리스트로 만들자
s =" ".join(map(str, extract_major))
s

'   Unnamed: 0                   major main_major\n0           0  mechanical engineering   tech_eng\n2           2  mechanical engineering   tech_eng      Unnamed: 0         major  main_major\n185           1  film studies  humanities\n227          43  film studies  humanities      Unnamed: 0       major  main_major\n124           2  psychology  social_sci\n156          34  psychology  social_sci'

In [20]:
ext_input_major_query = s.split()# str을 공란()으로 구분하여 리스트로 만들고, 이것을 비교(전공의 카테로리, 활동내용의 카테고리)할거임
ext_input_major_query

['Unnamed:',
 '0',
 'major',
 'main_major',
 '0',
 '0',
 'mechanical',
 'engineering',
 'tech_eng',
 '2',
 '2',
 'mechanical',
 'engineering',
 'tech_eng',
 'Unnamed:',
 '0',
 'major',
 'main_major',
 '185',
 '1',
 'film',
 'studies',
 'humanities',
 '227',
 '43',
 'film',
 'studies',
 'humanities',
 'Unnamed:',
 '0',
 'major',
 'main_major',
 '124',
 '2',
 'psychology',
 'social_sci',
 '156',
 '34',
 'psychology',
 'social_sci']

In [21]:
# 3. 이제 비교가 시작된다. 

#데이터 불러오기
data_act = pd.read_csv('activity_major_db.csv')
data_act[:10]

,activity_type,title,big_major_category_1,big_major_category_2,big_major_category_3
0,activity,team hbv,math_sci,math_sci,social_sci
1,activity,circle of women,humanities,social_sci,social_sci
2,activity,fine art/art schools and music for society,visual arts,music,NaN
3,activity,deca (distributive education clubs of america),business,math_sci,NaN
4,activity,society for american baseball research,math_sci,business,social_sci
5,activity,unite for sight,math_sci,math_sci,math_sci
6,activity,students for sensible drug policy,social_sci,social_sci,social_sci
7,activity,young americans for liberty,social_sci,humanities,social_sci
8,activity,timmy global health,math_sci,social_sci,math_sci
9,activity,stand now,social_sci,humanities,social_sci


In [22]:
act_title = data_act['title']  
act_title

0                                            team hbv
1                                     circle of women
2          fine art/art schools and music for society
3      deca (distributive education clubs of america)
4              society for american baseball research
                            ...                      
306                                             movie
307                                             video
308                                              film
309                                               sns
310                                         marketing
Name: title, Length: 311, dtype: object

In [23]:
len(act_title) #확인

311

In [24]:
data_act['title'][2].split() #확인.. 타이틀의 3번째 값 가져와본다. 

['fine', 'art/art', 'schools', 'and', 'music', 'for', 'society']

In [25]:
major1 = data_act['big_major_category_1'][6] #확인.. 잘됨
major1

'social_sci'

In [26]:
#입력데이터에서 추출한 활동내역(리스트)가 활동 타이틀에 있는지 확인한다.

n = 0
get_major = []
for t in act_title: # 311개의 활동 타이틀에서 한개씩 꺼내서 비교
    for j in activity_list: # 입력한 활동관련 설명문의 토큰을 하나씩 가져와서
        if n <= len(act_title)-1: #모든 타이틀을 검사한다.
            for m in data_act['title'][n].split(): #타이틀의 n번째 문장을 단어로 분해한 후 각 단어를 가져와서 활동관련 설명 단어와 비교한다.
                if j == m: # 매칭되는게 있다면, 해당 전공관련 주제를 추출해본다. 총 3개다. 
                    print('FIT')                  
                    major1 = data_act['big_major_category_1'][n]
                    get_major.append(major1)
                    major2 = data_act['big_major_category_2'][n]
                    get_major.append(major2)
                    major3 = data_act['big_major_category_3'][n]
                    get_major.append(major3)
                    n += 1
                else:
                    #print('NOT FIT')
                    #fitness.append('NOT FIT')
                    n += 1
                    
        else:
            pass

get_major #추출한 전공이다. 이 전공 카테고리와 희망전공 3개입력한 내용의 전공분야별 카테고리를 비교해 볼 것이다.

FIT
FIT


['humanities', 'social_sci', nan, 'social_sci', 'social_sci', nan]

df_fit_re = pd.DataFrame(get_major)
df_fit_re.columns = ['MAJOR']
list_fit_re = df_fit_re.drop_duplicates() #중복값 제거!!!! 결과 도출
list_fit_re

## 위 코드는 Activity Title과 학생의 활동내역을 비교하여 관련 전공을 추출하였음
# 다음 코드로는 입력전공 3개와 관련한 전공적합성을 비교분석한다.

In [27]:
major_list #재확인 .. 이것이 입력한 전공임 .. 

['mechanical engineering', 'film studies', 'psychology']

In [28]:
ext_input_major_query #major_list 으로부터 데이터베이스에서 추출한 관련 전공 데이터.... 이것을 비교할 것임

['Unnamed:',
 '0',
 'major',
 'main_major',
 '0',
 '0',
 'mechanical',
 'engineering',
 'tech_eng',
 '2',
 '2',
 'mechanical',
 'engineering',
 'tech_eng',
 'Unnamed:',
 '0',
 'major',
 'main_major',
 '185',
 '1',
 'film',
 'studies',
 'humanities',
 '227',
 '43',
 'film',
 'studies',
 'humanities',
 'Unnamed:',
 '0',
 'major',
 'main_major',
 '124',
 '2',
 'psychology',
 'social_sci',
 '156',
 '34',
 'psychology',
 'social_sci']

In [29]:
get_major # 입력한 활동내역을 토대로 분석한 결과 매칭 추출된 값

['humanities', 'social_sci', nan, 'social_sci', 'social_sci', nan]

In [30]:
#드디어 분석 fit analysis!!!!
#선택한 전공 3개,1개를 비교하여 데이터베이스 비교 분석하여 활동내역과 전공이 일치하는지 판단!
fit_result = []


# 수상내역이 데이터에 없을 경우 not sure. 그러나 수상점수는 계산이 가능함(first, winner 등의 단어를 확인해서)
if fit_list_str == None: #이 false 라면 not sure로 해야 함
    print ("not sure")
    fit_result.append('not sure')
else:
    for category in get_major: #분석 추출한 결과 내역들을 모두 단어로 쪼개어서 하나씩 추출해서 비교
        for major_item in ext_input_major_query:# 신청한 전공 3개와 연관된 주제단어를 가져와서
            if major_item  == category: # 전공이 있으면 fit
                print ("fit")
                fit_result.append('fit')

            else: # 원하는 전공 3개와 연관된 전공중 아무것도 매칭되는 것이 없으면 not sure
                    print ("not fit")
                    fit_result.append('not fit')

NameError: name 'fit_list_str' is not defined

In [ ]:
fit_result #데이터프레임으로
df_fit_re = pd.DataFrame(fit_result)
df_fit_re

In [ ]:
df_fit_re.columns = ['fit']
df_fit_re

In [ ]:
list_fit_re = df_fit_re.drop_duplicates() #중복값 제거!!!! 결과 도출, 여기서 FIT: 1 or NOT FIT : 2 or NOT SURE : 0 구분하면 됨
list_fit_re

In [ ]:
#조건문을 만들어서 결과를 출력해보자.
fit_anaysis_result_fin =[]
if 'fit' in list_fit_re: # fi이 있다면 NOT FIT 출력
    print("NOT FIT")
    fit_anaysis_result_fin.append('NOT FIT')
elif 'not fit' in list_fit_re: # not fit 이 있다면 , NOT FIT 출력
    print("NOT FIT")
    fit_anaysis_result_fin.append('FIT')
else:
    print("NOT SURE")
    fit_anaysis_result_fin.append('NOT SURE')
    
fit_anaysis_result_fin

## 1차 분석 끝!    이제 함수로 처리하여 10개의 입력을 받아서, 몇개의 fit이 나왔는지 카운트 하면 됨

## 이하코드는 이전에 작성한 것이지만 지금 활용하지 않음.. csv 파일로 저장하여 활용하였음.

In [ ]:
# major fit을 계산하려면 입력한 희망전공 3개를 예: ['mechanical engineering', ' computer science', ' bioligy']

# 각 전공을 다음 리스트에서 검색하여 매칭되는 값이 일치(여기서는 business이면) 하면 'FIT',  아니라면 ''NOT FIT 없으면 '?'로 표시


tech_eng_ = ['mechanical engineering', 'general Engineering', 'mechanical Engineering', 'Industrial engineering', 'bioengineering',
            'operations Research', 'materials Science', 'electrical Engineering', 'computer Science/engineering','computer engineering'
            'civil engineering', 'aerospace Engineering', 'chemical Engineering', 'environmental engineering', 'stem', 'steam', 'robotics', 
            'robots', 'computer', 'computers', 'computer programming', 'programming', 'coding', 'computing', 'mechanical', 'mechanics', 
            'mechanism', 'ai', 'artificial intelligence', 'sensor', 'environmental engineering', 'environmental technology', 
            'alternative energy', 'renewable energy', 'material science', 'biomedical', 'biomaterials', 'nanotech', 'pharmaceutical',
            'biomechanics', 'biomimicking', 'biomimicry', 'machine', 'bioengineering', 'chemical engineering', 'applied physics', 
            'operations research', 'civil engineering', 'aerospace', 'system', 'operations research', 'industrial engineering', 'computer Science'
            'invention', 'electrical', 'computational']

tech_eng = []
for i in tech_eng_:
    i_ = i.lower()
    tech_eng.append(i_)

math_sci_ = ['Agriculture', 'Physics', 'Biology', 'Biophysics', 'Biochemistry/Molecular Biology',
            'Mathematics', 'Applied Mathematics', 'Nutrition/Food Science', 'Genetics', 'Astronomy/Astrophysics',
            'Statistics', 'Chemistry', 'Neuroscience', 'Economics','Nursing','Pre-Med', 'Pre-Veterinary'
            'Pharmacy/Pre-Pharmacy', 'Pre-Dental', 'Pharmacy', 'Pre-Pharmacy', 'STEM', 'STEAM', 'Chemistry', 'physics',
            'biology', 'bio', 'biochem', 'biochemistry', 'science', 'math', 'mathematics', 'algorithm', 'biophysics', 
            'applied math', 'genetics', 'scientific', 'astronomy', 'astrophysics', 'statistics', 'neuroscience', 'dental',
            'doctor', 'physical', 'medical', 'scientific research', 'Biomaterials', 'material science', 'nutrition', 'clinical',
            'earth science', 'environmental science', 'ecology', 'ecological', 'veterinary', 'molecular', 'nanotech', 'biomedical',
            'pharmaceutical', 'food science, biotech', 'brain', 'botany', 'botanical', 'immunology', 'immune', 'zoology', 'biomechanics',
            'biomimicking', 'biomimicry']

math_sci = []
for j in math_sci_:
    j_ = j.lower()
    math_sci.append(j_)
    
social_sci_ = ['Economics Asian Studies', 'African Studies','Psychology', "Women's Studies",'Anthropology', 'Political Science/Government',
                'Political Science', 'Political Government', 'International Relations/Affairs', 'International Relations',
                'International Affairs', 'Public Policy/Affairs', 'Public Policy', 'Public Affairs', 'Public Health',
                'Environmental Science/Studies', 'Environmental Science', 'Environmental Studies', 'Urban Planning', 'Education',
                'Volunteer', 'community', 'community service', 'culture', 'equality', 'service',' social issue', 'social', 
                'society', 'economic', 'economy', 'SDGs', 'environment', 'policy', 'psychology', 'Asian studies', 'race relations', 
                'ethnicity', 'ethnic', 'women', 'anthropology', 'politic', 'international relations', 'international affairs',
                'diplomacy', 'diplomat', 'public policy', 'public health,' 'inequality', 'geography', 'geographic', 'linguistic', 
                 'sociology', 'culture', 'urban studies', 'diversity', 'LGBT', 'LGBTQ', 'sustainable', 'sustainability', 'debate',
                 'Middle East',' Middle Eastern Studies']

social_sci = []
for k in social_sci_:
    k_ = k.lower()
    social_sci.append(k_)

humanities_ = ['Archeology', 'Film Studies','German (Language and Literature)','Art History',
                'French (Language and Literature)','Spanish','Theology/Religious Studies',
                'Linguistics','History','English Language and Literature','Comparative Literature',
                'Philosophy','Classical Studies/Latin/Greek', 'English', 'literature', 'history', 'writing', 
              'heritage', 'essay', 'script', 'culture' ,'humanities', 'poem', 'poet', 'playwright', 'script',
              'poetry', 'literary', 'lit', 'writing', 'Spanish', 'German', 'Chinese', 'Italian', 'French', 'Russian', 'literature', 
              'Latin', 'Greek', 'anthropology', 'archaeology', 'Law', 'ethics', 'film studies']

humanities = []
for l in humanities_:
    l_ = l.lower()
    humanities.append(l_)
    

visualart_ = ['Film&Television', 'Film', 'Television', 'Design (Graphic, Industrial, Computer Graphics)','Graphic Design','Industral Design', 
             'Computer Graphics', 'Fashion Design', 'Photography', 'Architecture', 'Movie', 'visual', 'music', 'art', 'arts', 'performing', 
             'paint', 'painting',' visual arts', 'design', 'draw', 'drawing', 'graphic', 'architecture', 'architect', 'film', 'sculpture',
             'artwork', 'fine art']

visualart = []
for m in visualart_:
    m_ = m.lower()
    visualart.append(m_)
    

business_ = ['Business', 'Finance', 'Accounting', 'Marketing', 'Intl', 'Biz','International Business','Mgmt.', 'Info Systems', 
            'Hotel/Hospitality Management', 'Hotel Management', 'Hospitality Management', 'Business', 'start up', 'start-up',
            'entrepreneur', 'entrepreneurship', 'investment', 'invest', 'business plan', 'finance', 'marketing',' organizational behavior',
            'logistics', 'management', 'managing', 'social entrepreneur', 'social enterprise', 'accounting', 'managerial', 'MIS', 
            'Management Information Systems', 'Hotel management', 'hospitality']

business = []
for o in business_:
    o_ = o.lower()
    business.append(o_)
    

music_ = ['Music (Performance)', 'Music', 'Music Theory']

music = []
for p in music_:
    p_ = p.lower()
    music.append(p_)
    

performing_arts_ = ['performing arts','performance', 'dance', 'Movie', 'music', 'musical', 'theater', 'theatre', 'opera', 'thespian', 'song',
                   'sing', 'vocal', 'drama', 'orchestra', 'band', 'acting', 'acrobatics', 'ballet','circus', 'magic', 'mime', 'puppetry',
                   'ventriloquism', 'spoken word', 'stand-up comedy', 'Chinese opera', 'pansori', 'chamber music', 'cabaret', 
                   'jazz band', 'big band', 'Bharatanatyam', 'Gamelan semar pegulingan', 'Shōmyō', 'Odissi', 'Cantonese opera']

performing_arts = []
for q in performing_arts_:
    q_ = q.lower()
    performing_arts.append(q_)

communication_ = ['Documentary', 'SNS', 'Social Network', 'PR', 'public relations', 'advertisement', 'advertising', 'movie', 'film', 'news',
                 'journalism', 'newspaper', 'debate', 'speech', 'newspaper', 'media', 'communication']

communication = []
for r in communication_:
    r_ = r.lower()
    communication.append(r_)

In [ ]:
#비교 not fit 계산을 위해서 전공 리스트를 모두 합쳐보자
all_majors = tech_eng_ + math_sci_ + social_sci_ + humanities_ + visualart_ + business_ + music_ + performing_arts_ + communication_

In [ ]:
df_a = pd.DataFrame(tech_eng, columns=["major"])
df_a["main_major"] = "tech_eng"
df_a[:5]

In [ ]:
df_b = pd.DataFrame(math_sci, columns=["major"])
df_b["main_major"] = "math_sci"
df_b[:5]

In [ ]:
df_c = pd.DataFrame(social_sci, columns=["major"])
df_c["main_major"] = "social_sci"
df_c[:5]

In [ ]:
df_d = pd.DataFrame(humanities, columns=["major"])
df_d["main_major"] = "humanities"
df_d[:5]

In [ ]:
df_e = pd.DataFrame(business, columns=["major"])
df_e["main_major"] = "business"
df_e[:5]

In [ ]:
df_f = pd.DataFrame(music, columns=["major"])
df_f["main_major"] = "music"
df_f[:5]

In [ ]:
df_g = pd.DataFrame(performing_arts, columns=["major"])
df_g["main_major"] = "performing_arts"
df_g[:5]

In [ ]:
df_h = pd.DataFrame(communication, columns=["major"])
df_h["main_major"] = "communication"
df_h[:5]

In [ ]:
# 전공별 데이터와 주 전공을 하나의 데이터프레임으로 합친다. 
total_major_df = pd.concat([df_a,df_b,df_c,df_d,df_e,df_f,df_g,df_h])
total_major_df

In [ ]:
#데이터처리한 결과를 저장해놓는다. 
total_major_df.to_csv("total_major_fit_dataset.csv", mode='w')

In [ ]:
#이제 main_major에서 관련 전공을 선택하고, 어떤 분야의 main_major인지 매칭해서 fit, not fit, ? 를 구분한다.
major_list # 다시 확인. 그리고나서 이것을 total_major_df에서 검색을 한다.  매칭되면 main_major와 fit한지 판단.

In [ ]:
# 원하는 전공 리스트의 값을 데이터프레임 total_major_df에서 찾아보자
fit_analysis_list = []
for search_major in major_list:
    
    re__ = total_major_df[total_major_df['major'] == search_major]  # 데이터프레임에 찾고자하는 전공이 있다면
    print (re__) # 결과물을 리스트에 저장해보자
    fit_analysis_list.append(re__)



In [ ]:
fit_analysis_list

In [ ]:
#fit_analysis_list 를 문자로 만들어서 개별 리스트로 만들자
s =" ".join(map(str, fit_analysis_list))
s

In [ ]:
fit_list_str = s.split()# str을 공란()으로 쪼갬   
fit_list_str

In [ ]:
#드디어 분석 fit analysis!!!!
#선택한 전공 3개,1개를 비교하여 데이터베이스 비교 분석하여 활동내역과 전공이 일치하는지 판단!
fit_result = []


# 수상내역이 데이터에 없을 경우 not sure. 그러나 수상점수는 계산이 가능함(first, winner 등의 단어를 확인해서)
if fit_list_str == None: #이 false 라면 not sure로 해야 함
    print ("not sure")
    fit_result.append('not sure')
else:
    for category in get_major: #분석 추출한 결과 내역들을 모두 단어로 쪼개어서 하나씩 추출해서 비교
        for major_item in major_list:# 신청한 전공 3개와 연관된 주제단어를 가져와서
            if major_item  == category: # 전공이 있으면 fit
                print ("fit")
                fit_result.append('fit')

            else: # 원하는 전공 3개와 연관된 전공중 아무것도 매칭되는 것이 없으면 not sure
                    print ("not fit")
                    fit_result.append('not fit')

In [ ]:
fit_result #데이터프레임으로
df_fit_re = pd.DataFrame(fit_result)
df_fit_re

In [ ]:
df_fit_re.columns = ['fit']
df_fit_re

In [ ]:
list_fit_re = df_fit_re.drop_duplicates() #중복값 제거!!!! 결과 도출, 여기서 FIT: 1 or NOT FIT : 2 or NOT SURE : 0 구분하면 됨
list_fit_re

In [ ]:
'fit'in list_fit_re #확인해보고.. 데이터프레임에 fit이라는것이 있다면 true 로???

In [ ]:
'not fit' in list_fit_re # not fit이 있다면 flase이다. 반대인가?

In [ ]:
#조건문을 만들어서 결과를 출력해보자.
fit_anaysis_result_fin =[]
if 'fit' in list_fit_re: # fi이 있다면 NOT FIT 출력
    print("NOT FIT")
    fit_anaysis_result_fin.append('NOT FIT')
elif 'not fit' in list_fit_re: # not fit 이 있다면 , NOT FIT 출력
    print("NOT FIT")
    fit_anaysis_result_fin.append('FIT')
else:
    print("NOT SURE")
    fit_anaysis_result_fin.append('NOT SURE')
    
fit_anaysis_result_fin

1)데이터프레임의 각 행의 값을 가져와서, 각 단어를 분해(토큰화)하여, 리스트에 담고 : ['young' 'americans' 'for' 'liberty']
2)activity_list의 값을 위의 결과값과 비교하여, 존재한다면, MAJOR FIT을 비교하여 판단한다.

In [ ]:
title_item = data_act['title'][0] # 첫번재 값을 가져와서
t_item = title_item.split() 
t_item

In [ ]:
#2)activity_list의 값을 위의 결과값과 비교하여, 존재한다면
for i in activity_list:
    for k in t_item:
        if i == k:
            print('FIT')
        else:
            print('NOT FIT')